In [1]:
import os
import pandas as pd
from tabulate import tabulate

test_1_path = 'test_1'
test_2_path = 'test_2'
test_3_path = 'test_3'
test_4_path = 'test_4'  

def get_accuracy(folder_path):
  accuracy_dict = {}
  for model_folder in os.listdir(folder_path):
    model_path = os.path.join(folder_path, model_folder)
    if os.path.isdir(model_path):
      if os.path.exists(os.path.join(model_path, 'summary_metrics.csv')):
        summary_file = os.path.join(model_path, 'summary_metrics.csv')
        df = pd.read_csv(summary_file)
        accuracy = df.loc[df['Model'] == model_folder, 'Accuracy'].values[0]
        accuracy_dict[model_folder] = accuracy
  return accuracy_dict

test_1_accuracy = get_accuracy(test_1_path)
test_2_accuracy = get_accuracy(test_2_path)
test_3_accuracy = get_accuracy(test_3_path)
test_4_accuracy = get_accuracy(test_4_path)

df = pd.DataFrame({
  'Model': list(test_1_accuracy.keys()),
  'Test 1': list(test_1_accuracy.values()),
  'Test 1 (few-shot)': [test_3_accuracy.get(model, None) for model in test_1_accuracy.keys()],
  'Test 2': [test_2_accuracy.get(model, None) for model in test_1_accuracy.keys()],
  'Test 2 (few-shot)': [test_4_accuracy.get(model, None) for model in test_1_accuracy.keys()],
})

baseline_df = pd.read_csv('dataset/baseline.csv')
baseline_df = baseline_df[baseline_df['Class Name'] != 'Mean']

# Calculate TP as recall * (TP + FN) = recall * # Test Images
baseline_df['Recall'] = baseline_df['Recall'].str.rstrip('%').astype('float') / 100.0
baseline_df['# Test Images'] = baseline_df['# Test Images'].str.replace(',', '').astype('int')
baseline_df['True Positives'] = baseline_df['# Test Images'] * baseline_df['Recall']

# Calculate accuracy as TP / # Test Images
total_true_positives = baseline_df['True Positives'].sum()
total_test_images = baseline_df['# Test Images'].sum()
baseline_accuracy = total_true_positives / total_test_images

df.set_index('Model', inplace=True)
df.sort_index(inplace=True)

df['Test 1 (few-shot)'] = df['Test 1 (few-shot)'].fillna('-')

df = df.reindex([idx for idx in df.index if idx != 'gpt-4o-mini'] + ['gpt-4o-mini'])
df = df.reindex([idx for idx in df.index if idx != 'gpt-4o'] + ['gpt-4o'])

df.loc['Baseline'] = [f"{baseline_accuracy:.2%}", "-", "-", "-"]
df = df.reindex(['clip-vit-base-patch32'] + [idx for idx in df.index if idx != 'clip-vit-base-patch32'])

print(df.to_markdown())


| Model                     | Test 1   | Test 1 (few-shot)   | Test 2   | Test 2 (few-shot)   |
|:--------------------------|:---------|:--------------------|:---------|:--------------------|
| clip-vit-base-patch32     | 16.15%   | 21.41%              | 31.55%   | 35.46%              |
| clip-vit-base-patch16     | 25.64%   | 29.13%              | 28.70%   | 32.56%              |
| clip-vit-large-patch14    | 30.58%   | 31.71%              | 44.31%   | 47.64%              |
| siglip-base-patch16-512   | 48.71%   | 55.90%              | 68.19%   | 67.65%              |
| siglip-large-patch16-384  | 54.45%   | 53.49%              | 72.21%   | 70.65%              |
| siglip-so400m-patch14-384 | 53.86%   | 56.38%              | 70.55%   | 71.73%              |
| gpt-4o-mini               | 67.17%   | -                   | 64.43%   |                     |
| gpt-4o                    | 84.98%   | -                   | 89.32%   |                     |
| Baseline                  | 84.44%   |

In [3]:
def get_f1_scores(folder_path):
  f1_scores_dict = {}
  for model_folder in os.listdir(folder_path):
    model_path = os.path.join(folder_path, model_folder)
    if os.path.isdir(model_path):
      class_metrics_file = os.path.join(model_path, 'class_metrics.csv')
      df = pd.read_csv(class_metrics_file)
      f1_scores = df.set_index('class_name')['F1 Score'].to_dict()
      f1_scores_dict[model_folder] = f1_scores
  return f1_scores_dict

test_1_f1_scores = get_f1_scores(test_1_path)
test_1_df = pd.DataFrame.from_dict(test_1_f1_scores, orient='index')
test_1_df.drop(columns=['Macro', 'Micro'], inplace=True)
test_1_df.sort_index(inplace=True)
test_1_df.index.name = 'Model'
test_1_df = test_1_df.reindex(['clip-vit-base-patch32'] + [idx for idx in test_1_df.index if idx != 'clip-vit-base-patch32'])

# Read classes.csv to map IDs to Labels
classes_df = pd.read_csv('dataset/classes.csv')
id_to_label = dict(zip(classes_df['ID'], classes_df['Label']))

baseline_df = baseline_df[baseline_df['Class Name'] != 'Mean']
test_1_df.columns = [id_to_label.get(col, col) for col in test_1_df.columns]

# Append baseline F1-Score to test_1_df
baseline_f1_scores = baseline_df.groupby('Class Name')['F1-Score'].sum().to_dict()
test_1_df.loc['Baseline'] = [baseline_f1_scores.get(class_name, '-') for class_name in test_1_df.columns]

print("Test 1 F1 Scores:")
print(test_1_df.to_markdown())



Test 1 F1 Scores:
| Model                     | Antony of Padua   | Francis of Assisi   | Jerome   | John the Baptist   | Mary Magdalene   | Paul   | Peter   | Saint Dominic   | Saint Sebastian   | Virgin Mary   |
|:--------------------------|:------------------|:--------------------|:---------|:-------------------|:-----------------|:-------|:--------|:----------------|:------------------|:--------------|
| clip-vit-base-patch32     | 0.59%             | 34.61%              | 0.00%    | 19.18%             | 24.32%           | 0.00%  | 0.00%   | 12.67%          | 33.56%            | 10.52%        |
| clip-vit-base-patch16     | 3.33%             | 42.40%              | 0.00%    | 36.80%             | 25.81%           | 0.00%  | 0.00%   | 12.61%          | 40.53%            | 29.17%        |
| clip-vit-large-patch14    | 13.95%            | 64.89%              | 0.00%    | 27.86%             | 30.28%           | 6.35%  | 1.64%   | 16.29%          | 58.54%            | 35.29%        |
| 

In [4]:
print("\nTest 2 F1 Scores:")

test_2_f1_scores = get_f1_scores(test_2_path)
test_2_df = pd.DataFrame.from_dict(test_2_f1_scores, orient='index')
test_2_df.drop(columns=['Macro', 'Micro'], inplace=True)
test_2_df.sort_index(inplace=True)
test_2_df = test_2_df.reindex(['clip-vit-base-patch32'] + [idx for idx in test_2_df.index if idx != 'clip-vit-base-patch32'])
test_2_df.index.name = 'Model'

test_2_df.columns = [id_to_label.get(col, col) for col in test_2_df.columns]

test_2_df.loc['Baseline'] = [baseline_f1_scores.get(class_name, '-') for class_name in test_2_df.columns]
print(test_2_df.to_markdown())


Test 2 F1 Scores:
| Model                     | Antony of Padua   | Francis of Assisi   | Jerome   | John the Baptist   | Mary Magdalene   | Paul   | Peter   | Saint Dominic   | Saint Sebastian   | Virgin Mary   |
|:--------------------------|:------------------|:--------------------|:---------|:-------------------|:-----------------|:-------|:--------|:----------------|:------------------|:--------------|
| clip-vit-base-patch32     | 9.21%             | 17.71%              | 51.75%   | 27.59%             | 34.07%           | 22.90% | 22.33%  | 8.51%           | 23.24%            | 41.67%        |
| clip-vit-base-patch16     | 6.09%             | 5.22%               | 51.82%   | 36.76%             | 20.25%           | 30.00% | 32.56%  | 25.81%          | 29.05%            | 36.43%        |
| clip-vit-large-patch14    | 14.43%            | 36.99%              | 59.65%   | 47.78%             | 34.23%           | 32.79% | 46.58%  | 22.80%          | 32.74%            | 54.30%        |
|